In [50]:
# Imports

import matplotlib
import numpy as np
from enum import Enum
from collections import deque
from queue import PriorityQueue
from queue import Queue
from copy import copy, deepcopy
import sys
import random

In [51]:
# Coordinate Class
# Source: http://pythonfiddle.com/coordinate-class/

class Coordinate(object):
    
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def getX(self):
        return self.x
    def getY(self):
        return self.y
    def getCoordinates(self):
        return self.x, self.y

    def __str__(self):
        return '<' + str(self.getX()) + ',' + str(self.getY()) + '>'  
    def __unicode__(self):
        return '<' + str(self.getX()) + ',' + str(self.getY()) + '>'
    def __repr__(self):
        return '<' + str(self.getX()) + ',' + str(self.getY()) + '>'
    
    def __eq__(self, other):
        return self.y == other.y and self.x == other.x
    
    #https://stackoverflow.com/questions/9135759/java-hashcode-for-a-point-class
    def __hash__(self):
        result = self.x;
        result = 31 * result + self.y;
        return result;

In [52]:
# RoomElement Enum

class RoomElement(Enum):
    EMPTY = 0
    WALL = 1
    DOOR = 2
    HAZARD = 3
    EXIT = 4

In [53]:
# Helper Methods
def random_elements_of_list(list, N, K):
    newList = []
    indexes = random.sample(range(N), K)
    for index in indexes:
        newList.append(list[index])
    return newList

In [54]:
# Room Class

class Room():
    
    def __init__(self):
        self.map = np.array([[]])
        self.hazards = []
        self.originalMap = self.map
        
    def __init__(self, m):
        self.map = np.array(m)
        self.hazards = []
        self.originalMap = self.map
        
    def get_element_coordinates_of_map(self, roomElement):
        spots = []
        y = 0
        for row in self.map:
            x = 0
            for value in row:
                if value == roomElement.value:
                    coord = Coordinate(y, x)
                    spots.append(coord)
                x += 1   
            y += 1
        return spots
    
    def create_new_hazard(self):
        # do something to map
        room_empty_spots = self.get_element_coordinates_of_map(RoomElement.EMPTY)
        if len(room_empty_spots) < 1:
            return None
        room_hazard_spot = random_elements_of_list(room_empty_spots, len(room_empty_spots), 1)[0]
        self.map[room_hazard_spot.x][room_hazard_spot.y] = RoomElement.HAZARD.value
        self.hazards.append(Coordinate(room_hazard_spot.x, room_hazard_spot.y))
        
    def expand_existing_hazard(self):
        room_hazard_spots = self.get_element_coordinates_of_map(RoomElement.HAZARD)
        if len(room_hazard_spots) < 1:
            return None
        room_hazard_spot = random_elements_of_list(room_hazard_spots, len(room_hazard_spots), 1)[0]
        
        successors = []
        successors.append(Coordinate(room_hazard_spot.x - 1, room_hazard_spot.y))
        successors.append(Coordinate(room_hazard_spot.x + 1, room_hazard_spot.y))
        successors.append(Coordinate(room_hazard_spot.x, room_hazard_spot.y - 1))
        successors.append(Coordinate(room_hazard_spot.x, room_hazard_spot.y + 1))
        
        possibleExpansionSpots = []
        
        for c in successors:
            if c.x < 0 or c.y < 0 or c.x >= self.map.shape[0] or c.y >= self.map.shape[1]:
                continue
            if self.map[c.x][c.y] == RoomElement.WALL.value:
                continue
            if self.map[c.x][c.y] == RoomElement.HAZARD.value:
                continue
            if self.map[c.x][c.y] == RoomElement.DOOR.value:
                continue
            if self.map[c.x][c.y] == RoomElement.EXIT.value:
                continue
            possibleExpansionSpots.append(c)
            
            
        new_room_hazard_spot = random_elements_of_list(possibleExpansionSpots, len(possibleExpansionSpots), 1)[0]
        self.map[new_room_hazard_spot.x][new_room_hazard_spot.y] = RoomElement.HAZARD.value 
        self.hazards.append(Coordinate(new_room_hazard_spot.x, new_room_hazard_spot.y))
        
    
    def print_current_map_spot(self, c: Coordinate):
        mapCopy = deepcopy(self.map)
        mapCopy[c.x][c.y] = 9999
        return mapCopy
    
    def print_current_map_spots(self, mapSpots):
        mapCopy = deepcopy(self.map)
        for spot in mapSpots:
            mapCopy[spot.x][spot.y] = 9999
        return mapCopy
    
    def paths_to_a_goal(self, start: Coordinate, end: Coordinate):
        stack = Queue()
        visited = set()
        possiblePlans = []
        actions = []
        
        stack.put((start, actions))
        
        while not stack.empty():
            tupleObject = stack.get()
            coord = tupleObject[0]
            if coord.getCoordinates() in visited:
                continue
            
            actionListSoFar = tupleObject[1]
            
            visited.add(coord);
            if coord == end:
                possiblePlans.append(actionListSoFar)
                continue
            
            successors = []
            successors.append(Coordinate(coord.x - 1, coord.y))
            successors.append(Coordinate(coord.x + 1, coord.y))
            successors.append(Coordinate(coord.x, coord.y - 1))
            successors.append(Coordinate(coord.x, coord.y + 1))
            
            for c in successors:
                if c.x < 0 or c.y < 0 or c.x >= self.map.shape[0] or c.y >= self.map.shape[1]:
                    continue
                if self.map[c.x][c.y] == RoomElement.WALL.value:
                    continue
                if self.map[c.x][c.y] == RoomElement.HAZARD.value:
                    continue
                    
                
                if (not c in visited):
                    newActions = actionListSoFar + [c]
                    stack.put((c, newActions))
        
        return possiblePlans
    
    def best_path(self, paths):
        bestDistance = sys.maxsize
        bestPath = []
        
        for path in paths:
            unitsMoved = len(path)
            if (unitsMoved < bestDistance):
                bestDistance = unitsMoved
                bestPath = path
        
        return bestPath
    
    def reset_map(self):
        self.map = self.originalMap

In [55]:
# Person Class
class Person():
    
    def __init__(self):
        self.currentCoordinate = Coordinate(0, 0)
        self.path = []
        self.index = 0
        self.exited = False
        self.goal = Coordinate(0, 0)
        
    def __init__(self, c, p):
        self.currentCoordinate = c
        self.index = 0
        self.path = p
        self.exited = False
        self.goal = p[-1]
        
    def hasExited(self):
        if self.index >= len(self.path):
            self.exited = False
        if len(self.path) == 0:
            self.exited = True
        if (self.currentCoordinate == self.goal):
            self.exited = True
        return self.exited
    
    def get_current_coordinate(self):
        return self.currentCoordinate
        
    def next_spot_in_path(self):
        if (len(self.path) == 1):
            return self.path[0]
        if self.hasExited():
            return None
        if (self.index + 1 >= len(self.path)):
            return None
        nextSpot = self.path[self.index]
        return nextSpot
    
    def move_to_next_spot_in_path(self):
        if self.index >= len(self.path) - 1:
            self.exited = True
        if self.path[-1] == self.currentCoordinate:
            self.exited = True
        if not self.hasExited():
            if len(self.path) == 1:
                self.currentCoordinate = self.path[0]
            else:
                self.currentCoordinate = self.path[self.index]
        self.update_index(self.index + 1)
        
    def update_index(self, i):
        self.index = i
        
    def __str__(self):
        return 'Person: @' + str(self.currentCoordinate) + ', Path: ' + str(self.path) + ', Exited? ' + str(self.exited)  
    def __unicode__(self):
        return 'Person: @' + str(self.currentCoordinate) + ', Path: ' + str(self.path) + ', Exited? ' + str(self.exited)  
    def __repr__(self):
        return 'Person: @' + str(self.currentCoordinate) + ', Path: ' + str(self.path) + ', Exited? ' + str(self.exited)  

In [56]:
# EvacuationSimulation Class
class EvacuationSimulation():
    
    def __init__(self):
        self.room = Room()
        self.peopleCoordinates = []
        
    def __init__(self, r, p):
        self.room = r
        self.peopleCoordinates = p
        
    def get_path_based_on_current_coordinate(self, coordinate, room):
        paths = []
        exits = room.get_element_coordinates_of_map(RoomElement.EXIT)
        for exit in exits:
            somePaths = self.room.paths_to_a_goal(coordinate, exit)
            paths.extend(somePaths)
        bestPath = room.best_path(paths)
        return bestPath
    
    def everyone_has_exited(self, personList):
        for person in personList:
            if not person.hasExited():
                return False
        return True

    def simulate(self):
        personList = []
        for coordinate in self.peopleCoordinates:
            personPath = self.get_path_based_on_current_coordinate(coordinate, self.room)
            newPerson = Person(coordinate, personPath)
            personList.append(newPerson)
        
        print('Initialize simulation')
        time = 0
        print('Time: ' + str(time))
        print(self.room.print_current_map_spots(self.peopleCoordinates))
        print(personList)
        
        # Start Loop
        
        while not self.everyone_has_exited(personList):
            locationDict = {}
            for person in personList:
                if (person.next_spot_in_path() is None):
                    continue
                    '''
                    if (len(person.path) != 1):
                        continue
                    '''
                nextPlaceInPath = person.next_spot_in_path()
                print("Next spot: " + str(nextPlaceInPath))
                
                if nextPlaceInPath in locationDict and locationDict[nextPlaceInPath] is not None and len(locationDict[nextPlaceInPath]) > 1:
                    oldList = locationDict[nextPlaceInPath]
                    newList = oldList.append(person)
                    locationDict[nextPlaceInPath] = newList
                else:
                    locationDict[nextPlaceInPath] = [person]
                    
            
            print(locationDict)
            if len(locationDict.keys()) == 0:
                break
            
            for key in locationDict.keys():
                print(key)
                print(locationDict[key])
                if locationDict[key] is None:
                    continue
                
                if len(locationDict[key]) == 1:
                    # person can move on
                    personToMove = locationDict[key][0]
                    personToMove.move_to_next_spot_in_path()
                else:
                    # one random person moves forward
                    randomIndex = random.sample(range(len(locationDict[key])), 1)
                    persontoMove = locationDict[key][randomIndex]
                    personToMove.move_to_next_spot_in_path()
                    
            time = time + 1
            print('Time: ' + str(time))
            coordinates = []
            for person in personList:
                c = person.get_current_coordinate()
                coordinates.append(c)
            print('Coordinates')
            print(coordinates)
            print(self.room.print_current_map_spots(coordinates))
            
            
        # End Loop
        print('Done with simulation')
        
        
        
    
    def simulate_with_hazards(self):
        personList = []
        for coordinate in self.peopleCoordinates:
            personPath = self.get_path_based_on_current_coordinate(coordinate, self.room)
            newPerson = Person(coordinate, personPath)
            personList.append(newPerson)
        
        print('Initialize simulation')
        time = 0
        print('Time: ' + str(time))
        print(self.room.print_current_map_spots(self.peopleCoordinates))
        print(personList)
        
        # Start Loop
        
        while not self.everyone_has_exited(personList):
            
            randomNumber = random.randint(0, 10)
            print("Random Number: " + str(randomNumber))
            pathreevaluation = False
            
            if (randomNumber < 3):
                self.room.create_new_hazard()
                pathreevaluation = True
            if (randomNumber > 7):
                self.room.expand_existing_hazard()
                pathreevaluation = True
            print("Hazards: " + str(self.room.hazards))
            # People need to adjust pathing
            if pathreevaluation:
                for person in personList:
                    if person.hasExited():
                        continue
                    print("Before: " + str(person.path))
                    person.path = self.get_path_based_on_current_coordinate(person.currentCoordinate, self.room)
                    print("After: " + str(person.path))
                    person.index = 0
                    person.currentCoordinate = person.path[0]
                    self.exited = False
                    
                
            locationDict = {}
            for person in personList:
                if (person.next_spot_in_path() is None):
                    continue
                    '''
                    if (len(person.path) != 1):
                        continue
                    '''
                nextPlaceInPath = person.next_spot_in_path()
                print("Next spot: " + str(nextPlaceInPath))
                
                if nextPlaceInPath in locationDict and locationDict[nextPlaceInPath] is not None and len(locationDict[nextPlaceInPath]) > 1:
                    oldList = locationDict[nextPlaceInPath]
                    newList = oldList.append(person)
                    locationDict[nextPlaceInPath] = newList
                else:
                    locationDict[nextPlaceInPath] = [person]
                    
            
            print(locationDict)
            if len(locationDict.keys()) == 0:
                break
            
            for key in locationDict.keys():
                print(key)
                print(locationDict[key])
                if locationDict[key] is None:
                    continue
                
                if len(locationDict[key]) == 1:
                    # person can move on
                    personToMove = locationDict[key][0]
                    personToMove.move_to_next_spot_in_path()
                else:
                    # one random person moves forward
                    randomIndex = random.sample(range(len(locationDict[key])), 1)
                    persontoMove = locationDict[key][randomIndex]
                    personToMove.move_to_next_spot_in_path()
                    
            time = time + 1
            print('Time: ' + str(time))
            coordinates = []
            for person in personList:
                c = person.get_current_coordinate()
                coordinates.append(c)
            print('Coordinates')
            print(coordinates)
            print(self.room.print_current_map_spots(coordinates))
            
            
        # End Loop
        print('Done with simulation')

In [8]:
# ROOM 1
# One exit in a walled empty room.
room_1_map = [
    [1, 1, 1, 1, 1, 1, 1, 1, 4, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]
room_1 = Room(room_1_map)

room_1_empty_spots = room_1.get_element_coordinates_of_map(RoomElement.EMPTY)
room_1_empty_spots_subset = random_elements_of_list(room_1_empty_spots, len(room_1_empty_spots), int(len(room_1_empty_spots)/10))
print('Character Spots: ' + str(room_1_empty_spots_subset))

simulation1 = EvacuationSimulation(room_1, room_1_empty_spots_subset)
simulation1.simulate()

Character Spots: [<4,2>, <6,2>, <8,5>, <1,4>, <2,7>, <4,4>]
Initialize simulation
Time: 0
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0 9999    0    0    0    0    1]
 [   1    0    0    0    0    0    0 9999    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0 9999    0 9999    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0 9999    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0 9999    0    0    0    1]
 [   1    1    1    1    1    1    1    1    1    1]]
[Person: @<4,2>, Path: [<3,2>, <2,2>, <1,2>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False, Person: @<6,2>, Path: [<5,2>, <4,2>, <3,2>, <2,2>, <1,2>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False, Person: @<8,5>, Path: [<7,5>, <6,5>, <5,5>, <4,5>, <3,5>, <2,5>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False, Person: @<1,4>, Path: [<1,5>, 

In [12]:
simulation1.simulate_with_hazards()

Initialize simulation
Time: 0
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0 9999    0    0    0    0    1]
 [   1    0    0    0    0    0    0 9999    0    1]
 [   1    3    3    0    0    0    0    0    0    1]
 [   1    3 9999    0 9999    0    0    0    0    1]
 [   1    3    0    0    0    0    0    0    0    1]
 [   1    0 9999    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0 9999    0    0    0    1]
 [   1    1    1    1    1    1    1    1    1    1]]
[Person: @<4,2>, Path: [<4,3>, <3,3>, <2,3>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False, Person: @<6,2>, Path: [<5,2>, <4,2>, <4,3>, <3,3>, <2,3>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False, Person: @<8,5>, Path: [<7,5>, <6,5>, <5,5>, <4,5>, <3,5>, <2,5>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False, Person: @<1,4>, Path: [<1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False, Person: @<2,7>, 

After: [<0,8>]
Before: [<1,7>, <1,8>, <0,8>]
After: [<0,8>]
Before: [<3,4>, <2,4>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>]
After: [<1,7>, <1,8>, <0,8>]
Next spot: <1,5>
Next spot: <1,3>
Next spot: <2,5>
Next spot: <0,8>
Next spot: <0,8>
Next spot: <1,7>
{<1,5>: [Person: @<1,5>, Path: [<1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False], <1,3>: [Person: @<1,3>, Path: [<1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False], <2,5>: [Person: @<2,5>, Path: [<2,5>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False], <0,8>: [Person: @<0,8>, Path: [<0,8>], Exited? False], <1,7>: [Person: @<1,7>, Path: [<1,7>, <1,8>, <0,8>], Exited? False]}
<1,5>
[Person: @<1,5>, Path: [<1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False]
<1,3>
[Person: @<1,3>, Path: [<1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False]
<2,5>
[Person: @<2,5>, Path: [<2,5>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False]
<0,8>
[Person: @<0,8>, Path: [<0,8>], Exited? False]
<1,7>
[Person: @<1,7>, Path: [<1

After: [<0,8>]
Next spot: <0,8>
Next spot: <0,8>
Next spot: <0,8>
Next spot: <0,8>
Next spot: <0,8>
Next spot: <0,8>
{<0,8>: [Person: @<0,8>, Path: [<0,8>], Exited? True]}
<0,8>
[Person: @<0,8>, Path: [<0,8>], Exited? True]
Time: 15
Coordinates
[<0,8>, <0,8>, <0,8>, <0,8>, <0,8>, <0,8>]
[[   1    1    1    1    1    1    1    1 9999    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    3    0    0    1]
 [   1    3    3    0    0    0    0    0    0    1]
 [   1    3    3    0    0    0    0    0    0    1]
 [   1    3    0    0    0    0    0    0    0    1]
 [   1    3    0    0    0    3    0    0    0    1]
 [   1    0    3    3    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    1    1    1    1    1    1    1    1    1]]
Done with simulation


In [10]:
# ROOM 2
# Same as room 1 but the walking path is a square.
room_2_map = [
    [1, 1, 1, 1, 1, 1, 1, 1, 4, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]

room_2 = Room(room_2_map)

room_2_empty_spots = room_2.get_element_coordinates_of_map(RoomElement.EMPTY)
room_2_empty_spots_subset = random_elements_of_list(room_2_empty_spots, len(room_2_empty_spots), int(len(room_2_empty_spots)/10))
print('Character Spots: ' + str(room_2_empty_spots_subset))

simulation2 = EvacuationSimulation(room_2, room_2_empty_spots_subset)
simulation2.simulate()

Character Spots: [<5,1>, <8,3>]
Initialize simulation
Time: 0
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    1    1    1    1    1    1    0    1]
 [   1    0    1    1    1    1    1    1    0    1]
 [   1    0    1    1    1    1    1    1    0    1]
 [   1 9999    1    1    1    1    1    1    0    1]
 [   1    0    1    1    1    1    1    1    0    1]
 [   1    0    1    1    1    1    1    1    0    1]
 [   1    0    0 9999    0    0    0    0    0    1]
 [   1    1    1    1    1    1    1    1    1    1]]
[Person: @<5,1>, Path: [<4,1>, <3,1>, <2,1>, <1,1>, <1,2>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False, Person: @<8,3>, Path: [<8,4>, <8,5>, <8,6>, <8,7>, <8,8>, <7,8>, <6,8>, <5,8>, <4,8>, <3,8>, <2,8>, <1,8>, <0,8>], Exited? False]
Next spot: <4,1>
Next spot: <8,4>
{<4,1>: [Person: @<5,1>, Path: [<4,1>, <3,1>, <2,1>, <1,1>, <1,2>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Ex

In [11]:
# ROOM 3
# Same as Room 1 but with two exits.
room_3_map = [
    [1, 1, 1, 1, 1, 1, 1, 1, 4, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 4, 1, 1, 1, 1, 1, 1, 1, 1]
]

room_3 = Room(room_3_map)

room_3_empty_spots = room_3.get_element_coordinates_of_map(RoomElement.EMPTY)
room_3_empty_spots_subset = random_elements_of_list(room_3_empty_spots, len(room_3_empty_spots), int(len(room_3_empty_spots)/10))
print('Character Spots: ' + str(room_3_empty_spots_subset))

simulation3 = EvacuationSimulation(room_3, room_3_empty_spots_subset)
simulation3.simulate()

Character Spots: [<7,5>, <8,1>, <6,8>, <1,5>, <3,6>, <5,8>]
Initialize simulation
Time: 0
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0 9999    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0 9999    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [   1    0    0    0    0 9999    0    0    0    1]
 [   1 9999    0    0    0    0    0    0    0    1]
 [   1    4    1    1    1    1    1    1    1    1]]
[Person: @<7,5>, Path: [<8,5>, <8,4>, <8,3>, <8,2>, <8,1>, <9,1>], Exited? False, Person: @<8,1>, Path: [<9,1>], Exited? False, Person: @<6,8>, Path: [<5,8>, <4,8>, <3,8>, <2,8>, <1,8>, <0,8>], Exited? False, Person: @<1,5>, Path: [<1,6>, <1,7>, <1,8>, <0,8>], Exited? False, Person: @<3,6>, Path: [<2,6>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False, Person: @<5,8>, Path: [<4,8>,

{<9,1>: [Person: @<9,1>, Path: [<9,1>], Exited? True]}
<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 86
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1 9999    0    0    0    0    0    0    0    1]
 [   1 9999    1    1    1    1    1    1    1    1]]
Next spot: <9,1>
{<9,1>: [Person: @<9,1>, Path: [<9,1>], Exited? True]}
<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 87
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [  

 [   1 9999    1    1    1    1    1    1    1    1]]
Next spot: <9,1>
{<9,1>: [Person: @<9,1>, Path: [<9,1>], Exited? True]}
<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 171
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1 9999    0    0    0    0    0    0    0    1]
 [   1 9999    1    1    1    1    1    1    1    1]]
Next spot: <9,1>
{<9,1>: [Person: @<9,1>, Path: [<9,1>], Exited? True]}
<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 172
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    1 

<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 275
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1 9999    0    0    0    0    0    0    0    1]
 [   1 9999    1    1    1    1    1    1    1    1]]
Next spot: <9,1>
{<9,1>: [Person: @<9,1>, Path: [<9,1>], Exited? True]}
<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 276
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [  

{<9,1>: [Person: @<9,1>, Path: [<9,1>], Exited? True]}
<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 400
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1 9999    0    0    0    0    0    0    0    1]
 [   1 9999    1    1    1    1    1    1    1    1]]
Next spot: <9,1>
{<9,1>: [Person: @<9,1>, Path: [<9,1>], Exited? True]}
<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 401
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [

...

 [   1 9999    1    1    1    1    1    1    1    1]]
Next spot: <9,1>
{<9,1>: [Person: @<9,1>, Path: [<9,1>], Exited? True]}
<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 4055
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1 9999    0    0    0    0    0    0    0    1]
 [   1 9999    1    1    1    1    1    1    1    1]]
Next spot: <9,1>
{<9,1>: [Person: @<9,1>, Path: [<9,1>], Exited? True]}
<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 4056
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    

 [   1 9999    1    1    1    1    1    1    1    1]]
Next spot: <9,1>
{<9,1>: [Person: @<9,1>, Path: [<9,1>], Exited? True]}
<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 4140
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1 9999    0    0    0    0    0    0    0    1]
 [   1 9999    1    1    1    1    1    1    1    1]]
Next spot: <9,1>
{<9,1>: [Person: @<9,1>, Path: [<9,1>], Exited? True]}
<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 4141
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    

 [   1 9999    1    1    1    1    1    1    1    1]]
Next spot: <9,1>
{<9,1>: [Person: @<9,1>, Path: [<9,1>], Exited? True]}
<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 4226
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1 9999    0    0    0    0    0    0    0    1]
 [   1 9999    1    1    1    1    1    1    1    1]]
Next spot: <9,1>
{<9,1>: [Person: @<9,1>, Path: [<9,1>], Exited? True]}
<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 4227
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    

[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1 9999    0    0    0    0    0    0    0    1]
 [   1 9999    1    1    1    1    1    1    1    1]]
Next spot: <9,1>
{<9,1>: [Person: @<9,1>, Path: [<9,1>], Exited? True]}
<9,1>
[Person: @<9,1>, Path: [<9,1>], Exited? True]
Time: 4335
Coordinates
[<8,1>, <9,1>, <1,8>, <1,8>, <1,8>, <1,8>]
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0   

KeyboardInterrupt: 

In [ ]:
# Checkpoint Notes
'''
1. Implement a hazard. - Rohan
Fire can expand at random rates (stochastic).

2. Different rates of people. - Rohan
Define that? Each person has different average walking speed and standard deviation.
Subclasses for People, each with statistics. And different behavior for collisions.

3. Randomized map generator - Rishab & Dheeraj

4. Best optimal room evaluation (evaluation score) - Rishab & Dheeraj
    How to do this? Put notes here.

''';

In [68]:
# Randomize Map Method 1
'''
Randomly generate NxM array (25 > N > 5, 25 > M > 5) 
where the room is bounded by walls and all of the inner space is empty.
However, one space in the wall that isn't a corner will be a door.
''';

import numpy as np
import random

N = random.randint(5,25)
M = random.randint(5,25)
room = []
N = 5
M = 6
for i in range(0, N):
    print(i)
    if i == 0 or i == N - 1:
        firstRow = list()
        for j in range(0, M):
            firstRow.append(1)
        room.append(firstRow)        
    else:
        for j in range(0, M):
            firstElement = 1
            lastElement = 1
            row = list()
            row.append(1)
            for k in range(0, M-2):
                row.append(0)
            row.append(1)
        room.append(row)

doorCoordinateX = random.choice([0, N-1])
doorCoordinateY = random.randint(1, M-2)


door2CoordinateX = random.randint(1, N-2)
door2CoordinateY = random.choice([0, M-1])

door = (doorCoordinateX, doorCoordinateY)
door2 = (door2CoordinateX, door2CoordinateY)

door_min = random.choice([door, door2])
room[door_min[0]][door_min[1]] = 4

room = Room(room)
room_3_empty_spots = room.get_element_coordinates_of_map(RoomElement.EMPTY)
room_3_empty_spots_subset = random_elements_of_list(room_3_empty_spots, len(room_3_empty_spots), int(len(room_3_empty_spots)/10))
# print('Character Spots: ' + str(room_3_empty_spots_subset))

simulation3 = EvacuationSimulation(room, room_3_empty_spots_subset)
simulation3.simulate()



0
1
2
3
4
Initialize simulation
Time: 0
[[   1    1    1    1    1    1]
 [   1    0 9999    0    0    1]
 [   1    0    0    0    0    1]
 [   1    0    0    0    0    1]
 [   1    4    1    1    1    1]]
[Person: @<1,2>, Path: [<2,2>, <3,2>, <3,1>, <4,1>], Exited? False]
Next spot: <2,2>
{<2,2>: [Person: @<1,2>, Path: [<2,2>, <3,2>, <3,1>, <4,1>], Exited? False]}
<2,2>
[Person: @<1,2>, Path: [<2,2>, <3,2>, <3,1>, <4,1>], Exited? False]
Time: 1
Coordinates
[<2,2>]
[[   1    1    1    1    1    1]
 [   1    0    0    0    0    1]
 [   1    0 9999    0    0    1]
 [   1    0    0    0    0    1]
 [   1    4    1    1    1    1]]
Next spot: <3,2>
{<3,2>: [Person: @<2,2>, Path: [<2,2>, <3,2>, <3,1>, <4,1>], Exited? False]}
<3,2>
[Person: @<2,2>, Path: [<2,2>, <3,2>, <3,1>, <4,1>], Exited? False]
Time: 2
Coordinates
[<3,2>]
[[   1    1    1    1    1    1]
 [   1    0    0    0    0    1]
 [   1    0    0    0    0    1]
 [   1    0 9999    0    0    1]
 [   1    4    1    1    1    1]]
Ne

In [69]:
# Randomize Map Method 2
'''
Randomly generate NxM array (25 > N > 5, 25 > M > 5) 
where the room is bounded by walls and all of the inner space is empty.
However, multiple doors will exist in the in the wall (6 > D > 1) (none of which isn't a corner).
''';

import numpy as np
import random

N = random.randint(5,25)
M = random.randint(5,25)
room = []
N = 6
M = 5

for i in range(0, N):
    if i == 0 or i == N - 1:
        firstRow = list()
        for j in range(0, M):
            firstRow.append(1)
        room.append(firstRow)        
    else:
        for j in range(0, M):
            firstElement = 1
            lastElement = 1
            row = list()
            row.append(1)
            for k in range(0, M-2):
                row.append(0)
            row.append(1)
        room.append(row)

D = random.randint(1,6)
for i in range(0, D):
    
    doorCoordinateX = random.randint(1, N-2)
    doorCoordinateY = random.choice([0, M-1])


    door2CoordinateX = random.choice([0, N-1])
    door2CoordinateY = random.randint(1, M-2)

    door = (doorCoordinateX, doorCoordinateY)
    door2 = (door2CoordinateX, door2CoordinateY)
    

    door_min = random.choice([door, door2])
#     print(door_min)
    room[door_min[0]][door_min[1]] = 4

room = Room(room)
room_3_empty_spots = room.get_element_coordinates_of_map(RoomElement.EMPTY)
room_3_empty_spots_subset = random_elements_of_list(room_3_empty_spots, len(room_3_empty_spots), int(len(room_3_empty_spots)/10))
print('Character Spots: ' + str(room_3_empty_spots_subset))

simulation3 = EvacuationSimulation(room, room_3_empty_spots_subset)
simulation3.simulate()



Character Spots: [<3,2>]
Initialize simulation
Time: 0
[[   1    1    4    4    1]
 [   1    0    0    0    1]
 [   1    0    0    0    4]
 [   1    0 9999    0    1]
 [   1    0    0    0    1]
 [   1    1    1    1    1]]
[Person: @<3,2>, Path: [<2,2>, <1,2>, <0,2>], Exited? False]
Next spot: <2,2>
{<2,2>: [Person: @<3,2>, Path: [<2,2>, <1,2>, <0,2>], Exited? False]}
<2,2>
[Person: @<3,2>, Path: [<2,2>, <1,2>, <0,2>], Exited? False]
Time: 1
Coordinates
[<2,2>]
[[   1    1    4    4    1]
 [   1    0    0    0    1]
 [   1    0 9999    0    4]
 [   1    0    0    0    1]
 [   1    0    0    0    1]
 [   1    1    1    1    1]]
Next spot: <1,2>
{<1,2>: [Person: @<2,2>, Path: [<2,2>, <1,2>, <0,2>], Exited? False]}
<1,2>
[Person: @<2,2>, Path: [<2,2>, <1,2>, <0,2>], Exited? False]
Time: 2
Coordinates
[<1,2>]
[[   1    1    4    4    1]
 [   1    0 9999    0    1]
 [   1    0    0    0    4]
 [   1    0    0    0    1]
 [   1    0    0    0    1]
 [   1    1    1    1    1]]
{}
Done with 

In [ ]:
# Randomize Map Method 3
'''
Randomly generate NxM array (25 > N > 5, 25 > M > 5) 
where the room is bounded by walls and all of the inner space is empty.
However, one doors will exist in the in the wall that is multiple units long (consecutive D's in the wall),
and none of the D's can exist in any corner.
''';

In [ ]:
# Randomize Map Method 4
'''
Randomly generate NxM array (25 > N > 5, 25 > M > 5) 
where the room is bounded by walls and all of the inner space is empty.
However, random columns of walls exist that jut out from the walls. 
A person would still need to be able to get to an exist.
'''
# Example:
'''
11111
10001
10001
10101
10101
11111
''';